<a href="https://colab.research.google.com/github/ralphmartynward/ironhack_06_song-recommender/blob/master/song_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initiate Google Drive connection

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lab 1: Hot songs from billboard
https://github.com/isg75/lab-web-scraping-single-page

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_hot100():
    url = "https://www.billboard.com/charts/hot-100/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    song_elements = soup("div", class_="o-chart-results-list-row-container")
    #print("song_elements:", song_elements)
    
    songs = []
    for element in song_elements:
        song = {}
        song["title"] = element.find("h3").text.strip()
        song["artist"] = element.find("h3").find_next("span").text.strip()
        songs.append(song)
    
    #print("songs:", songs)
    
    hot100_df = pd.DataFrame(songs)
    return hot100_df

# Save the DataFrame to a CSV file
hot100_df.to_csv('../data/hot100_songs.csv', index=False)

hot100_df = scrape_hot100()
hot100_df.head()

,title,artist
0,Kill Bill,SZA
1,Last Night,Morgan Wallen
2,Flowers,Miley Cyrus
3,Princess Diana,Ice Spice & Nicki Minaj
4,Ella Baila Sola,Eslabon Armado X Peso Pluma


In [ ]:
# Ralph test 
# -> saving this on a database instead of a csv
# The goal being that we could host the database on a cloud service (such as Firebase) and call from there instead
!pip install tinydb
import os
from tinydb import TinyDB, Query
db_path = "/content/drive/My Drive/ironhack/song_recommender/data/hot100.json"
db = TinyDB(db_path)
db.insert_multiple(hot100_df.to_dict('records'))
hot100_df = pd.DataFrame(db.all())
hot100_df.head()

# Lab 2: Not hot songs
https://github.com/isg75/lab-not-hot-songs

In [ ]:
import pandas as pd

In [145]:
# using this Kaggle dataset example https://www.kaggle.com/datasets/mrmorj/dataset-of-songs-in-spotify
not_hot_songs = pd.read_csv('/content/drive/My Drive/ironhack/song_recommender/data/tracks_features.csv')

In [135]:
not_hot_songs.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [ ]:
not_hot_songs.shape

(1204025, 24)

In [146]:
not_hot_songs.drop(['album', 'artist_ids', 'track_number', 'disc_number', 'duration_ms', 'release_date', 'year', 'album_id', 'explicit'], axis=1, inplace=True)

In [137]:
not_hot_songs.isna().sum()

id                  0
name                0
artists             0
danceability        0
energy              0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64

In [147]:
not_hot_songs['artists']=[i[2:-2] for i in not_hot_songs['artists'] if i[0] =='[']

In [ ]:
# maybe we should drop duplicates here too

In [148]:
not_hot_songs.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,Rage Against The Machine,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,Rage Against The Machine,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,Rage Against The Machine,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,Rage Against The Machine,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,Rage Against The Machine,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0


In [149]:
not_hot_songs = not_hot_songs.to_csv("/content/drive/My Drive/ironhack/song_recommender/data/not_hot_songs_features.csv", index=False)

In [140]:
songs= not_hot_songs.drop(not_hot_songs.sample(frac=.998).index)
songs.shape

(2408, 14)

In [141]:
songs = songs.to_csv("/content/drive/My Drive/ironhack/song_recommender/data/songs.csv", index=False)

In [128]:
nothot= pd.read_csv('/content/drive/My Drive/ironhack/song_recommender/data/songs.csv')

In [129]:
nothot.head()


,id,name,album_id,artists,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,5OiJAVPCmvhQwAmCYFe6iI,Soul Love,4EEyCnj2fZu1eInXpLfZZW,Daryl Hall & John Oates,False,0.652,0.618,-6.049,0,0.0778,0.20000,0.000005,0.0418,0.971,171.844,4.0
1,3PiVLjJVEtOWndg8Ulknif,Rock & Roll,6yn5Mylr8HsH9PVro7Fo22,Johnny Winter,False,0.394,0.706,-12.147,1,0.0556,0.07000,0.006270,0.1480,0.563,141.695,4.0
2,3tuC2PqYsBplcBKKbl4X10,Dark Center Of The Universe,5SbcupirEParczvqqyUPXL,Modest Mouse,True,0.246,0.826,-3.505,1,0.0590,0.00808,0.000008,0.0675,0.240,192.578,4.0
3,06bAIcZ9aqV6kLEbtXPwLi,Never Let Go,3yqFs6Bf5DCyGnnJM2RR9B,In the Machine,False,0.332,0.798,-7.262,1,0.0493,0.00001,0.002910,0.0563,0.346,117.686,4.0
4,71z0nJT7qEgt66SKegC30a,Pray Your Gods,3UeshcanaZ1xd90cgmQmch,Toad The Wet Sprocket,False,0.558,0.213,-12.848,1,0.0286,0.83900,0.007540,0.1270,0.127,111.893,4.0


In [ ]:
# make this cleaning step of lab 2 into a function instead !

# Lab 3: Spotify API for features
https://github.com/isg75/lab-spotify-api

In [ ]:
! pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import sys
sys.path.append('/content/drive/My Drive/ironhack/song_recommender/') # so that the credentials are fetched from the root and hidden
from config import client_id, client_secret

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [143]:
import pandas as pd 
hot100_df = pd.read_csv('/content/drive/My Drive/ironhack/song_recommender/data/hot100_songs.csv')

In [ ]:
# find spotify id of each song in "hot" (sp.search()) https://spotipy.readthedocs.io/en/2.22.1/
def get_spotify_id(song_title, artist_name):
    query = f"{song_title} artist:{artist_name}"
    results = sp.search(q=query, type="track", limit=1)
    
    if results["tracks"]["total"] > 0:
        return results["tracks"]["items"][0]["id"]
    else:
        return None


hot100_df['id'] = hot100_df.apply(lambda row: get_spotify_id(row['Song'], row['Artist']), axis=1)

# improve this by printing status per song searched (finding song / found song)

In [105]:
hot100_df['id'].isna().sum()

30

In [ ]:
# Not the best results, 70%, not sure if there is a better way to search ?
# drop those rows otherwise as they are null

In [106]:
hot100_df = hot100_df.dropna(subset=['id'])

In [ ]:
# If we had more than 100 IDs to search, it's best to put it in a list ready for the next step
# And use chunks 

In [107]:
# get song features for each song (sp.audio_features())
def get_audio_features(spotify_id):
    if spotify_id is None:
        return None
    else:
        return sp.audio_features([spotify_id])[0]

hot100_df['features'] = hot100_df['id'].apply(get_audio_features)


In [ ]:
hot100_df.head()

,Song,Artist,id,features
0,Last Night,Morgan Wallen,59uQI0PADDKeE6UZDTJEe8,"{'danceability': 0.517, 'energy': 0.675, 'key'..."
1,Flowers,Miley Cyrus,0yLdNVWF3Srea0uzk55zFn,"{'danceability': 0.707, 'energy': 0.681, 'key'..."
2,Princess Diana,Ice Spice & Nicki Minaj,None,None
3,Ella Baila Sola,Eslabon Armado X Peso Pluma,None,None
4,Creepin',"Metro Boomin, The Weeknd & 21 Savage",2dHHgzDwk4BJdRwy9uXhTO,"{'danceability': 0.715, 'energy': 0.62, 'key':..."


In [109]:
# json format, not ideal, we'd need to beautify this into columns
# only get these features 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness' ,'valence', 'tempo', 'duration_ms', 'time_signature', 'year'
feature_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
for feature in feature_columns:
    hot100_df[feature] = hot100_df['features'].apply(lambda x: x[feature] if x is not None else None)


In [110]:
hot100_df.rename(columns={'Song': 'name'}, inplace=True)
hot100_df.rename(columns={'Artist': 'artists'}, inplace=True)
hot100_df = hot100_df[['id', 'name', 'artists', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]

In [111]:
hot100_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,59uQI0PADDKeE6UZDTJEe8,Last Night,Morgan Wallen,0.517,0.675,6,-5.382,1,0.0357,0.45900,0.000000,0.1510,0.518,203.853,4
1,0yLdNVWF3Srea0uzk55zFn,Flowers,Miley Cyrus,0.707,0.681,0,-4.325,1,0.0668,0.06320,0.000005,0.0322,0.646,117.999,4
4,2dHHgzDwk4BJdRwy9uXhTO,Creepin',"Metro Boomin, The Weeknd & 21 Savage",0.715,0.620,1,-6.005,0,0.0484,0.41700,0.000000,0.0822,0.172,97.950,4
5,1s7oOCT8vauUh01PbJD6ps,Calm Down,Rema & Selena Gomez,0.799,0.802,11,-5.196,1,0.0371,0.42900,0.001280,0.1710,0.811,107.008,4
7,7aRCf5cLOFN1U7kvtChY1G,Search & Rescue,Drake,0.817,0.440,10,-8.482,0,0.0734,0.06030,0.000001,0.3300,0.544,142.024,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1yHVHoz6Ny29gbbWJYVnFt,Heaven,Niall Horan,0.571,0.761,5,-3.485,1,0.0261,0.07080,0.000000,0.3310,0.681,91.991,4
94,5kB6WithhB8Iv08EMNmAbt,Memory Lane,Old Dominion,0.645,0.800,6,-4.272,1,0.0363,0.01890,0.000011,0.0858,0.375,118.977,4
95,4sx6NRwL6Ol3V6m9exwGlQ,Love Again,The Kid LAROI,0.662,0.398,11,-6.691,0,0.0275,0.71600,0.000000,0.1110,0.468,107.001,4
96,3oZ6dlSfCE9gZ55MGPJctc,'98 Braves,Morgan Wallen,0.488,0.670,2,-5.759,1,0.0268,0.09690,0.000220,0.2570,0.484,142.139,4


id	name	artists	**explicit**	danceability	energy	loudness	mode	speechiness	acousticness	instrumentalness	liveness	valence	tempo	time_signature

In [112]:
# after adding the features, save the dataframes (hot and not-hot) into csv files 
hot100_df = hot100_df.to_csv("/content/drive/My Drive/ironhack/song_recommender/data/hot100_df_features.csv", index=False)

# Lab 4: Clustering songs
https://github.com/isg75/lab-clustering-songs

In [6]:
import pandas as pd 
hot_songs = pd.read_csv('/content/drive/My Drive/ironhack/song_recommender/data/hot100_df_features.csv')
not_hot_songs = pd.read_csv('/content/drive/My Drive/ironhack/song_recommender/data/not_hot_songs_features.csv')

In [ ]:
# In this lesson, we will introduce unsupervised learning using K-means and DBSCAN methods to perform a simple clustering task.
# In this lesson, we will introduce the Gaussian Mixtures and Agglomerative Clustering methods to perform a simple clustering task.

# Storytelling 
Marketing presentation, no code, USP, blablabla

In [ ]:
# What is storytelling? Is the art of explaining a problem and the solution to it through a story. This story has several elements to consider:

# - The audience (boss, CEO, CTO, your mother,...). Each of them will need a different level of technical details.
# - The history script
# - The arc

In [ ]:
# Test on the list of songs from Trello

- Overloading - Mavins https://open.spotify.com/track/4lVEVXrlQlXPdYWMGTwrFe?si=OToCDPusQ6iAGEtcjKcP0w&context=spotify%3Asearch%3Aoverd
- Love me like you do - Ellie Goulding https://open.spotify.com/track/7jtnrvoeBjkp3uQ4wtK9ue?autoplay=true
- Unchained Melody - Righteous Brothers https://open.spotify.com/track/1jFhnVoJkcB4lf9tT0rSZS
- Quater past midnight - Bastille https://open.spotify.com/track/1etiUDkISHELzQGMY79ryt?si=1763c54575dd4b46
- Over the Rainbow - Israel Kamakawiwo'ole https://open.spotify.com/track/4yDjzVhXig9tfO7Zv46FE8?si=kfJQE4wxRbiuHBpNX4Dhqg
- Soso by Omah Lay https://open.spotify.com/track/1wADwLSkYhrSmy4vdy6BRn?si=b0MwDeZlTECbG-JW_v6SEg
- Igor Krutoy You are in my September https://open.spotify.com/track/45S1TeYStrAk7GUktFUiq
- Talking Heads - This Must Be The Place https://open.spotify.com/track/6aBUnkXuCEQQHAlTokv9or?si=44c083e9b2df40b2
- Mother of Sky - Can https://open.spotify.com/track/4cZM4JzqnMleplEHo7zNBa
- Spongebob Squarepants https://open.spotify.com/track/2LBpcqDEbzf8NfCa6VJWRs
- Lose yourself - Eminem https://open.spotify.com/track/7MJQ9Nfxzh8LPZ9e9u68Fq
- Elva Hsiao - CAPPUCCINO https://open.spotify.com/track/3hCyGYmytCmkExrFmT4I7w

In [ ]:
def read_song_and_artist():
    title_s = input("Please Enter the song title: ").lower()
    artist_s = input("Please Enter the artist title: ").lower()
    
    return artist_s,title_s

In [ ]:
# plot song features on a spider chart

In [ ]:
# from the recommended songs, show a link to spotify or a allow a preview of the song (similar to Tinder)
# generate a playlist of similar songs even, so the user can add it to his library? 

Moving forward

Convert to script, put in docker container, upload to heroku with link to github repo

Set up cron for weekly chart updates 

Deploy db on firebase, fetch features with API on the fly 